In [10]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [12]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
class FizzBuzz():
    def __init__(self, length=30):
        self.length = length  # 程序需要执行的序列长度
        self.array = tf.Variable([str(i) for i in range(1, length+1)], dtype=tf.string, trainable=False)  # 最后程序返回的结果
        self.graph = tf.while_loop(self.cond, self.body, [1, self.array],)   # 对每一个值进行循环判断

    def run(self):
        with tf.Session() as sess:
            tf.global_variables_initializer().run()
            return sess.run(self.graph) 
    
    def cond(self, i, _):
        return (tf.less(i, self.length+1)) # 判断是否是最后一个值
    
    def body(self, i, _):
        flow = tf.cond(tf.equal(tf.mod(i, 15), 0), lambda: tf.assign(self.array[i - 1], 'FizzBuzz'),
                lambda: tf.cond(tf.equal(tf.mod(i, 3), 0), lambda: tf.assign(self.array[i - 1], 'Fizz'),
                        lambda: tf.cond(tf.equal(tf.mod(i, 5), 0), lambda: tf.assign(self.array[i - 1], 'Buzz'),
                                lambda: self.array)
                                                   )
                                                    )
        return (tf.add(i, 1), flow)
        
if __name__ == '__main__':
    fizzbuzz = FizzBuzz(length=50)
    ix, array = fizzbuzz.run()
    print(array)

[b'1' b'2' b'Fizz' b'4' b'Buzz' b'Fizz' b'7' b'8' b'Fizz' b'Buzz' b'11'
 b'Fizz' b'13' b'14' b'FizzBuzz' b'16' b'17' b'Fizz' b'19' b'Buzz' b'Fizz'
 b'22' b'23' b'Fizz' b'Buzz' b'26' b'Fizz' b'28' b'29' b'FizzBuzz' b'31'
 b'32' b'Fizz' b'34' b'Buzz' b'Fizz' b'37' b'38' b'Fizz' b'Buzz' b'41'
 b'Fizz' b'43' b'44' b'FizzBuzz' b'46' b'47' b'Fizz' b'49' b'Buzz']


In [13]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

n = 100000
x = tf.constant(list(range(n)))
c = lambda i, x: i < n
b = lambda i, x: (tf.compat.v1.Print(i + 1, [i]), tf.compat.v1.Print(x + 1,[i], "x:"))
i, out = tf.while_loop(c, b, (0, x))
with tf.compat.v1.Session() as sess:
    print(sess.run(i))  # prints [0] ... [9999]
    # The following line may increment the counter and x in parallel.
    # The counter thread may get ahead of the other thread, but not the
    # other way around. So you may see things like
    # [9996] x:[9987]
    # meaning that the counter thread is on iteration 9996,
    # while the other thread is on iteration 9987
    print(sess.run(out).shape)

100000
(100000,)


In [3]:
import tensorflow as tf 
tf.compat.v1.disable_eager_execution()

a = tf.compat.v1.get_variable(name = "ii", dtype=tf.int32, shape=[], initializer=tf.ones_initializer())
n = tf.constant(10)
def cond(a, n):
    return  a< n
def body(a, n):
    a = a + 1
    return a, n
graph = tf.while_loop(cond, body, [a, n])
with tf.compat.v1.Session() as sess:
    tf.compat.v1.global_variables_initializer().run()
    a, res = sess.run(graph)
    print(a)

10


In [1]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
i = tf.constant(0)
acc = tf.constant(0)
cond = lambda i, _: tf.less(i, 10)
body = lambda i, acc: (tf.add(i, 1), tf.add(acc, i))
graph = tf.while_loop(cond, body, [i, acc])
with tf.compat.v1.Session() as sess:
    _, acc = sess.run(graph)
    print(acc)


45


In [29]:
import cv2
from imagededup.methods import CNN

def valid_url(url_string, encoding_map, start_sec = 20, end_sec = 40):
    # 视频抽帧+提取特征 大约需要11.9秒左右 （20张）
    video_capture.open(url_string)
    total_frames = video_capture.get(7)
    fps = video_capture.get(5)
    duration = total_frames // fps
    
    if duration >= end_sec:
        
        for sec in range(start_sec,end_sec):
            new_url_string = '{}\t{}'.format(url_string, sec)
            frame_num = sec * fps
            video_capture.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
            _, image = video_capture.read()
            image = cv2.resize(image, (224, 224),interpolation=cv2.INTER_CUBIC)
            feat_vec = cnn_encoder.encode_image(image_array= image)
            encoding_map[new_url_string] = feat_vec[0]
            #print(feat_vec.shape)
            #cv2.imshow('image',image)
            cv2.imwrite(new_url_string+'.jpg', image)  # save frame as JPEG file
            #break
            
        return url_string
    else:
        return 'Error: Cannot open video url: {}'.format(url_string)
    

In [30]:
%%time 
url_string = 'http://aqiniudl.tangdou.com/202005/3167C3677B9ECEAD33E0372055533CB4-10.mp4'
url_string2 = 'http://aqiniudl.tangdou.com/202005/B17A19A7D35E7B70B97D8D3D0B1F52F2-10.mp4'
video_capture = cv2.VideoCapture()
encoding_map = {}
cnn_encoder = CNN()
valid_url(url_string, encoding_map)
valid_url(url_string2, encoding_map)

2020-07-16 10:27:25,036: INFO Initialized: MobileNet pretrained on ImageNet dataset sliced at last conv layer and added GlobalAveragePooling


CPU times: user 20.2 s, sys: 2.88 s, total: 23.1 s
Wall time: 34.8 s


'http://aqiniudl.tangdou.com/202005/B17A19A7D35E7B70B97D8D3D0B1F52F2-10.mp4'

In [31]:
%%time
duplicates = cnn_encoder.find_duplicates(encoding_map=encoding_map,
                                             min_similarity_threshold=0.90,
                                             scores=True)

2020-07-16 10:28:11,659: INFO Start: Calculating cosine similarities...
2020-07-16 10:28:11,669: INFO End: Calculating cosine similarities.


CPU times: user 5.06 ms, sys: 3.05 ms, total: 8.11 ms
Wall time: 13.3 ms


In [35]:
cnt = 0
for key in duplicates.keys():
    pre_fix, suffix = key.split('\t')
#     print('*'*20)
#     print(pre_fix)
    self_filter = [ele for ele in duplicates[key] if pre_fix not in ele[0]]
    if self_filter:
        cnt += 1
print(cnt/2)

15.0


In [1]:
import json
import redis    
hostname = '10.42.158.47'
port = 6379
r = redis.Redis(host=hostname, port=port)
print(r.get('url_{}'.format(1500678022281)))
print(r.get('url_{}'.format(1500677989235)))
# def get_urls(vid):
#     # vid = '8982780'
#     key = 'url_{}'.format(vid)
#     res = json.loads(r.get(key))
#     pic = 'http://aimg.tangdou.com'+res['pic']
#     if '!' in pic:
#         pic = pic.split('!')[0]
#     return [vid, pic]
# print(get_urls(1500656814419))

b'{"pic": "/public/video/2020/1116/20201116184748_2893555.jpg!260", "videourl": "202011/481E8D63-CEA5-7D48-3F28-3EFAD83D9A64", "vid": 1500678022281, "mp3url": ""}'
b'{"pic": "/public/video/2020/1114/20201114011810_5957255.jpg!260", "videourl": "202011/189ABD42-24BC-B071-702B-9D9861C07F2C", "vid": 1500677989235, "mp3url": ""}'


In [ ]:
import json
from datetime import datetime
import time
import redis
import cv2
hostname = '10.42.158.47'
port = 6379
r = redis.Redis(host=hostname, port=port)

video_capture = cv2.VideoCapture()
def is_open(url):
    if video_capture.open(url):
        return url
    else:
        url = url.replace("-10", "-20")
        if video_capture.open(url):
            return url
        else:
            url = url.replace("-20.mp4", ".mp4")
            if video_capture.open(url):
                return url
            else:
                return None

# def get_urls(vid):
#     key = 'url_{}'.format(vid)
#     try:
#         res = json.loads(r.get(key))
#     except Exception as e:
#         print('ERROR:The key cannot find in redis')
#         return None
#     video = 'http://aqiniudl.tangdou.com/'+res['videourl']+'-10.mp4'
#     video = is_open(video)
#     if not video:
#         print('the video url of {} is invalid!'.format(vid))
#         return video
#     return video
def get_urls(vid):
    key = 'url_{}'.format(vid)
    try:
        res = json.loads(r.get(key))
    except Exception as e:
        print('ERROR:', 'The key cannot find in redis')
        return [None, None]
    pic = 'http://aimg.tangdou.com'+res['pic']
    video = 'http://aqiniudl.tangdou.com/'+res['videourl']+'-10.mp4'
    video = is_open(video)
    if '!' in pic:
        pic = pic.split('!')[0]
    if not video:
        return [None, pic]
    return [video, pic]
big_dict = {}
def convert_ts(createtime):
    dateobj = datetime.strptime(createtime, '%Y-%m-%d %H:%M:%S.%f')
    t = dateobj.timetuple()
    timeStamp = int(time.mktime(t))
    return timeStamp

with open('./vid_createtime', 'r', encoding = 'utf-8') as f:
    for line in f:
        line_json = json.loads(line)
        try:
            big_dict[line_json['vid']] = convert_ts(line_json['createtime'])
        except Exception as e:
            print(line_json['vid'], e)
print('createtime down')

with open('./out_final_bk','r' , encoding='utf-8') as w, open('./new_out_final','w', encoding='utf-8') as w1, open('./PM_out_final','w', encoding='utf-8') as w2:
    cnt = -0
    for line2 in w:
        cnt += 1
        print(cnt)
        detail_json = {}
        parse = json.loads(line2)
        org_vid = parse['id']
        org_video, org_img  = get_urls(org_vid)
        org_vid_time = big_dict[org_vid]
        detail_json['query'] = {'vid':org_vid,'img':org_img,'video':org_video,'createtime':org_vid_time}


        img_reprint_vids = parse['profileinfo']['img_reprint_vidlist']
        # 时间戳小的为原创视频
        detail_json['candidates'] = []
        for x in img_reprint_vids:
            tmp_time = big_dict[x]
            video_url, img_url  = get_urls(x)
            # 如果待选视频的创建时间 小于 请求视频的创建时间，则说明请求视频是盗播视频，否则为原创

            detail_json['candidates'].append({'vid':x,'img':img_url,'video':video_url,'createtime':tmp_time})
            if tmp_time < org_vid_time:
                pass
            else:
                parse['profileinfo']['img_reprint_flag'] = 0
                break
        w1.write(json.dumps(parse)+'\n')
        w2.write(json.dumps(detail_json)+'\n')

In [25]:
import json
from datetime import datetime
import time
import redis
import cv2
hostname = '10.42.158.47'
port = 6379
r = redis.Redis(host=hostname, port=port)

video_capture = cv2.VideoCapture()
def is_open(url):
    if video_capture.open(url):                 
        return url
    else:
        url = url.replace("-10", "-20")
        if video_capture.open(url):
            return url
        else:
            url = url.replace("-20.mp4", ".mp4")
            if video_capture.open(url):
                return url
            else:
                print('No url is valid!')
                return None

def get_urls(vid):
    # vid = '8982780'
    key = 'url_{}'.format(vid)
    print(key)
    try:
        res = json.loads(r.get(key))
    except Exception as e:
        print('ERROR:', 'The key cannot find in redis')
        return [None, None]
    pic = 'http://aimg.tangdou.com'+res['pic']
    video = 'http://aqiniudl.tangdou.com/'+res['videourl']+'-10.mp4'
    video = is_open(video)
    if '!' in pic:
        pic = pic.split('!')[0]
    if not video:
        return [None, pic]
    return [video, pic]

In [ ]:
%%time 
file = '/Users/jiangcx/Documents/tmp'
with open(file, 'r') as f:
    for line in f:
        vid = json.loads(line)['vid']
        video , pic = get_urls(vid)
        print(video)

In [59]:
import pandas as pd
file = '/Users/jiangcx/Documents/PM_out_final'
tmplist = []
with open(file, 'r') as f:
    for line in f:
        res = json.loads(line)
        vid = res['query']['vid']
        img = res['query']['img']
        video = res['query']['video']
        tmplist.append([str(vid), video, img])
        for x in res['candidates']:
            tmplist.append([str(x['vid']),x['video'], x['img']])
        tmplist.append(['',''])
        tmplist.append(['',''])
        
df = pd.DataFrame(tmplist)
df.to_csv('./res.csv')

In [2]:
abc = {'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t20': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t21', 0.9765388422942951), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t20', 1.0000000000000009), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t21', 0.9765388422942951), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t20', 0.9664486817137026), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t21', 0.9588598557276725)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t21': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t20', 0.9765388422942951), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t20', 0.9765388422942951), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t21', 1.0), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t20', 0.9436723184425178), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t21', 0.9646406199659256)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.967515245483085), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9545908407436745), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9510320172184293), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9487988626846922), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9453426610746947), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9420269084343285), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 1.0), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.967515245483085), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9545908407436745), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9510320172184293), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9487988626846922), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9453426610746947), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9420269084343285), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9491519198590459), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9351239718210458), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9084738077169975), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9171167196672914), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9089798950449235), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9106606043748955), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9119457062701004)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.967515245483085), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9818346198552483), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9649034425821205), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.966910036964164), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9616057998149854), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9597457445108959), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.967515245483085), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 1.000000000000001), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9818346198552483), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9649034425821205), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.966910036964164), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9616057998149854), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9597457445108959), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9400363642026343), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9560708161785081), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9305019784237264), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.92839501489237), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9257817614256327), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9301000890342855), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9268973376026303)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9545908407436745), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9818346198552483), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9757896514216114), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9760264456471854), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9701891850576712), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.968908924500756), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9545908407436745), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9818346198552483), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 1.0000000000000009), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9757896514216114), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9760264456471854), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9701891850576712), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.968908924500756), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9291973085733651), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9495744800017343), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9434390895444374), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9373351899592067), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9309343154802342), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9368971590351027), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9322183793696175)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9510320172184293), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9649034425821205), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9757896514216114), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9813168481356757), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9798131589928769), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9678316161260265), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9510320172184293), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9649034425821205), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9757896514216114), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 1.0000000000000007), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9813168481356757), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9798131589928769), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9678316161260265), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9272896067376721), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9413088348547333), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9341697105988589), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9489661025120988), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9358871617154356), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9422774697418275), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9374026542847339)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9487988626846922), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.966910036964164), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9760264456471854), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9813168481356757), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9862494521384304), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9787822380407533), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9487988626846922), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.966910036964164), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9760264456471854), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9813168481356757), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9862494521384304), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9787822380407533), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9296208229415251), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9452322099265886), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9397803629458887), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9453139022289001), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9515441646365497), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9530722903217728), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9496321905336094)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9453426610746947), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9616057998149854), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9701891850576712), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9798131589928769), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9862494521384304), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.982430499112139), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9453426610746947), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9616057998149854), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9701891850576712), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9798131589928769), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9862494521384304), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.982430499112139), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.921649955436247), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9360003821012137), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9283951159869825), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9392071640747245), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9395799123508236), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9524461287113019), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9451253443641043)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9420269084343285), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9597457445108959), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.968908924500756), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9678316161260265), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9787822380407533), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.982430499112139), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9420269084343285), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9597457445108959), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.968908924500756), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9678316161260265), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9787822380407533), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.982430499112139), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9234526654131376), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9411918862872091), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9344789106443988), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9382183991985195), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9398913499278009), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9527468905013836), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9569073020467556)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t29': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30', 0.9602142625588338), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t29', 1.0000000000000002), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30', 0.9602142625588338), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t29', 0.9749872228647614), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9361024691663535)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t29', 0.9602142625588338), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9329775016329104), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9002383409722086), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t29', 0.9602142625588338), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30', 0.9999999999999997), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9329775016329104), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9002383409722086), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t29', 0.9418503817589083), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9727964551823969), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9129105029679255)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30', 0.9329775016329104), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9662461941334398), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9602653270963092), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.96018658990498), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9601679493737644), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9610760196231306), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30', 0.9329775016329104), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9662461941334398), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9602653270963092), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.96018658990498), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9601679493737644), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9610760196231306), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9314439003567179), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9718049183382965), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9385714067723657), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9387688286138868), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9391945464479733), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9368265497597607), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.938042874780895)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9662461941334398), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9837549904627184), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9850403789422094), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.975302226950986), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9720601921271065), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9662461941334398), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 1.0000000000000002), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9837549904627184), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9850403789422094), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.975302226950986), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9720601921271065), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9079345003475547), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9583071856743355), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9686259404537378), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9660352858307433), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9633912545363514), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9571694600729512), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9570082710573644)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30', 0.9002383409722086), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9602653270963092), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9837549904627184), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9811495107350076), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.977244443843203), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.973117515305824), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30', 0.9002383409722086), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9602653270963092), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9837549904627184), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9811495107350076), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.977244443843203), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.973117515305824), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9083020645080476), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9498034721821738), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9539787386005629), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9661357152692859), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9563006068196144), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9524802100124091), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9520292446367055)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.96018658990498), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9850403789422094), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9811495107350076), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9844114308710198), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9753468523378317), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.96018658990498), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9850403789422094), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9811495107350076), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9844114308710198), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9753468523378317), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9046881958011269), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9551670741574686), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9645524740167776), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9660379136704926), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9697019009972268), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9612475627035132), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9602304292367366)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9601679493737644), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.975302226950986), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.977244443843203), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9844114308710198), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9860145261877805), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9601679493737644), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.975302226950986), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.977244443843203), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9844114308710198), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 1.0000000000000016), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9860145261877805), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9062776627607819), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9547850670800789), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9529711788078387), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.958930654419969), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9626388749816811), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9678330355473781), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9654679871617633)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9610760196231306), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9720601921271065), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.973117515305824), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9753468523378317), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9860145261877805), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9610760196231306), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9720601921271065), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.973117515305824), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9753468523378317), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9860145261877805), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9999999999999996), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9089780621125239), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9548422445650534), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9483997729129185), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9520979134980109), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9539353277901782), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9618731414101533), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9687344950837457)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t37': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t38', 0.9726189135005472), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t39', 0.9556185567745769), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t37', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t38', 0.9726189135005472), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t39', 0.9556185567745769), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t37', 0.9678946256910758), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t38', 0.9492862901829382), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t39', 0.9285383458413126)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t38': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t37', 0.9726189135005472), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t39', 0.9753692658584407), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t37', 0.9726189135005472), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t38', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t39', 0.9753692658584407), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t37', 0.9575510242255703), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t38', 0.9703131167279909), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t39', 0.9482410342776318)], 'http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t39': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t37', 0.9556185567745769), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t38', 0.9753692658584407), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t37', 0.9556185567745769), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t38', 0.9753692658584407), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t39', 1.0000000000000002), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t37', 0.9511900537357373), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t38', 0.9629753703589079), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t39', 0.9721080561167931)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t20': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t20', 1.0000000000000009), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t21', 0.9765388422942951), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t21', 0.9765388422942951), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t20', 0.9664486817137026), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t21', 0.9588598557276725)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t21': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t20', 0.9765388422942951), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t21', 1.0), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t20', 0.9765388422942951), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t20', 0.9436723184425178), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t21', 0.9646406199659256)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 1.0), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.967515245483085), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9545908407436745), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9510320172184293), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9487988626846922), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9453426610746947), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9420269084343285), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.967515245483085), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9545908407436745), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9510320172184293), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9487988626846922), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9453426610746947), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9420269084343285), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9491519198590459), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9351239718210458), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9084738077169975), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9171167196672914), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9089798950449235), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9106606043748955), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9119457062701004)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.967515245483085), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 1.000000000000001), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9818346198552483), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9649034425821205), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.966910036964164), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9616057998149854), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9597457445108959), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.967515245483085), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9818346198552483), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9649034425821205), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.966910036964164), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9616057998149854), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9597457445108959), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9400363642026343), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9560708161785081), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9305019784237264), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.92839501489237), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9257817614256327), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9301000890342855), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9268973376026303)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9545908407436745), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9818346198552483), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 1.0000000000000009), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9757896514216114), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9760264456471854), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9701891850576712), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.968908924500756), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9545908407436745), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9818346198552483), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9757896514216114), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9760264456471854), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9701891850576712), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.968908924500756), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9291973085733651), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9495744800017343), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9434390895444374), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9373351899592067), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9309343154802342), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9368971590351027), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9322183793696175)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9510320172184293), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9649034425821205), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9757896514216114), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 1.0000000000000007), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9813168481356757), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9798131589928769), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9678316161260265), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9510320172184293), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9649034425821205), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9757896514216114), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9813168481356757), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9798131589928769), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9678316161260265), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9272896067376721), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9413088348547333), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9341697105988589), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9489661025120988), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9358871617154356), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9422774697418275), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9374026542847339)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9487988626846922), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.966910036964164), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9760264456471854), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9813168481356757), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9862494521384304), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9787822380407533), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9487988626846922), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.966910036964164), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9760264456471854), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9813168481356757), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9862494521384304), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9787822380407533), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9296208229415251), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9452322099265886), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9397803629458887), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9453139022289001), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9515441646365497), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9530722903217728), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9496321905336094)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9453426610746947), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9616057998149854), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9701891850576712), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9798131589928769), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9862494521384304), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.982430499112139), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9453426610746947), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9616057998149854), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9701891850576712), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9798131589928769), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9862494521384304), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.982430499112139), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.921649955436247), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9360003821012137), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9283951159869825), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9392071640747245), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9395799123508236), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9524461287113019), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9451253443641043)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9420269084343285), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9597457445108959), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.968908924500756), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9678316161260265), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9787822380407533), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.982430499112139), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9420269084343285), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9597457445108959), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.968908924500756), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9678316161260265), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9787822380407533), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.982430499112139), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9234526654131376), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9411918862872091), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9344789106443988), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9382183991985195), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9398913499278009), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9527468905013836), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9569073020467556)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t29': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t29', 1.0000000000000002), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30', 0.9602142625588338), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30', 0.9602142625588338), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t29', 0.9749872228647614), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9361024691663535)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t29', 0.9602142625588338), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30', 0.9999999999999997), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9329775016329104), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9002383409722086), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t29', 0.9602142625588338), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9329775016329104), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9002383409722086), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t29', 0.9418503817589083), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9727964551823969), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9129105029679255)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30', 0.9329775016329104), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9662461941334398), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9602653270963092), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.96018658990498), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9601679493737644), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9610760196231306), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30', 0.9329775016329104), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9662461941334398), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9602653270963092), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.96018658990498), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9601679493737644), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9610760196231306), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9314439003567179), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9718049183382965), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9385714067723657), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9387688286138868), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9391945464479733), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9368265497597607), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.938042874780895)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9662461941334398), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 1.0000000000000002), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9837549904627184), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9850403789422094), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.975302226950986), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9720601921271065), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9662461941334398), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9837549904627184), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9850403789422094), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.975302226950986), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9720601921271065), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9079345003475547), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9583071856743355), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9686259404537378), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9660352858307433), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9633912545363514), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9571694600729512), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9570082710573644)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30', 0.9002383409722086), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9602653270963092), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9837549904627184), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9811495107350076), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.977244443843203), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.973117515305824), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30', 0.9002383409722086), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9602653270963092), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9837549904627184), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9811495107350076), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.977244443843203), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.973117515305824), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9083020645080476), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9498034721821738), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9539787386005629), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9661357152692859), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9563006068196144), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9524802100124091), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9520292446367055)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.96018658990498), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9850403789422094), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9811495107350076), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9844114308710198), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9753468523378317), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.96018658990498), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9850403789422094), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9811495107350076), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9844114308710198), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9753468523378317), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9046881958011269), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9551670741574686), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9645524740167776), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9660379136704926), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9697019009972268), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9612475627035132), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9602304292367366)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9601679493737644), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.975302226950986), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.977244443843203), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9844114308710198), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 1.0000000000000016), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9860145261877805), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9601679493737644), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.975302226950986), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.977244443843203), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9844114308710198), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9860145261877805), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9062776627607819), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9547850670800789), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9529711788078387), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.958930654419969), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9626388749816811), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9678330355473781), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9654679871617633)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9610760196231306), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9720601921271065), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.973117515305824), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9753468523378317), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9860145261877805), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9999999999999996), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9610760196231306), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9720601921271065), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.973117515305824), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9753468523378317), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9860145261877805), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9089780621125239), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9548422445650534), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9483997729129185), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9520979134980109), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9539353277901782), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9618731414101533), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9687344950837457)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t37': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t37', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t38', 0.9726189135005472), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t39', 0.9556185567745769), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t38', 0.9726189135005472), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t39', 0.9556185567745769), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t37', 0.9678946256910758), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t38', 0.9492862901829382), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t39', 0.9285383458413126)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t38': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t37', 0.9726189135005472), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t38', 1.0000000000000004), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t39', 0.9753692658584407), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t37', 0.9726189135005472), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t39', 0.9753692658584407), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t37', 0.9575510242255703), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t38', 0.9703131167279909), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t39', 0.9482410342776318)], 'http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t39': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t37', 0.9556185567745769), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t38', 0.9753692658584407), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t39', 1.0000000000000002), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t37', 0.9556185567745769), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t38', 0.9753692658584407), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t37', 0.9511900537357373), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t38', 0.9629753703589079), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t39', 0.9721080561167931)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t20': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t20', 0.9664486817137026), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t21', 0.9436723184425178), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t20', 0.9664486817137026), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t21', 0.9436723184425178), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t21', 0.9751654571876353)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t21': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t20', 0.9588598557276725), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t21', 0.9646406199659256), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t20', 0.9588598557276725), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t21', 0.9646406199659256), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t20', 0.9751654571876353)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9491519198590459), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9400363642026343), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9291973085733651), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9272896067376721), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9296208229415251), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.921649955436247), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9234526654131376), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9491519198590459), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9400363642026343), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9291973085733651), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9272896067376721), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9296208229415251), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.921649955436247), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9234526654131376), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9765229123639702), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9607787093489811), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.958914699961778), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9587685794321341), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9572857036420732), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9538537506084557)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9351239718210458), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9560708161785081), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9495744800017343), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9413088348547333), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9452322099265886), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9360003821012137), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9411918862872091), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9351239718210458), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9560708161785081), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9495744800017343), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9413088348547333), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9452322099265886), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9360003821012137), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9411918862872091), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9765229123639702), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9796119509870681), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9716671182490222), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9698816487432775), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9705072221229458), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9670699187161881)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9084738077169975), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9305019784237264), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9434390895444374), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9341697105988589), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9397803629458887), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9283951159869825), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9344789106443988), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9084738077169975), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9305019784237264), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9434390895444374), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9341697105988589), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9397803629458887), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9283951159869825), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9344789106443988), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9607787093489811), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9796119509870681), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.981223160483698), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9762421500638099), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9755005032605898), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9706384703405834)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9171167196672914), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.92839501489237), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9373351899592067), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9489661025120988), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9453139022289001), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9392071640747245), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9382183991985195), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9171167196672914), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.92839501489237), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9373351899592067), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9489661025120988), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9453139022289001), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9392071640747245), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9382183991985195), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.958914699961778), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9716671182490222), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.981223160483698), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9816984398546024), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9806672268273319), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9773222475395629)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9089798950449235), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9257817614256327), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9309343154802342), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9358871617154356), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9515441646365497), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9395799123508236), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9398913499278009), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9089798950449235), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9257817614256327), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9309343154802342), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9358871617154356), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9515441646365497), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9395799123508236), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9398913499278009), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9587685794321341), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9698816487432775), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9762421500638099), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9816984398546024), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9863134037901574), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9835810595412817)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9106606043748955), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9301000890342855), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9368971590351027), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9422774697418275), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9530722903217728), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9524461287113019), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9527468905013836), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9106606043748955), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9301000890342855), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9368971590351027), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9422774697418275), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9530722903217728), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9524461287113019), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9527468905013836), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9572857036420732), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9705072221229458), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9755005032605898), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9806672268273319), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9863134037901574), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28', 0.9869325463034782)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t28': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t22', 0.9119457062701004), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t23', 0.9268973376026303), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t24', 0.9322183793696175), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t25', 0.9374026542847339), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t26', 0.9496321905336094), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t27', 0.9451253443641043), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t28', 0.9569073020467556), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t22', 0.9119457062701004), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t23', 0.9268973376026303), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t24', 0.9322183793696175), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t25', 0.9374026542847339), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t26', 0.9496321905336094), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t27', 0.9451253443641043), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t28', 0.9569073020467556), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t22', 0.9538537506084557), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t23', 0.9670699187161881), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t24', 0.9706384703405834), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t25', 0.9773222475395629), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t26', 0.9835810595412817), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t27', 0.9869325463034782)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t29': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t29', 0.9749872228647614), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30', 0.9418503817589083), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t29', 0.9749872228647614), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30', 0.9418503817589083), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9519152411053851), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9021967975774545)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t29', 0.9361024691663535), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30', 0.9727964551823969), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9314439003567179), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9079345003475547), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9083020645080476), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9046881958011269), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9062776627607819), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9089780621125239), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t29', 0.9361024691663535), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30', 0.9727964551823969), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9314439003567179), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9079345003475547), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9083020645080476), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9046881958011269), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9062776627607819), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9089780621125239), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t29', 0.9519152411053851), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9390220707693655), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9055626339848731), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9126602155010046), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9108404984302801), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9145415186576021), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.912488702544453)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t30', 0.9129105029679255), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9718049183382965), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9583071856743355), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9498034721821738), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9551670741574686), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9547850670800789), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9548422445650534), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t30', 0.9129105029679255), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9718049183382965), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9583071856743355), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9498034721821738), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9551670741574686), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9547850670800789), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9548422445650534), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t29', 0.9021967975774545), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9390220707693655), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9673788127257743), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9643890726645791), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9664114008404745), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9661398549474679), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9668487798273079)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9385714067723657), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9686259404537378), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9539787386005629), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9645524740167776), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9529711788078387), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9483997729129185), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9385714067723657), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9686259404537378), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9539787386005629), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9645524740167776), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9529711788078387), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9483997729129185), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9055626339848731), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9673788127257743), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9849195487881457), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9859085412552211), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9760612426667575), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.975341424324142)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9387688286138868), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9660352858307433), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9661357152692859), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9660379136704926), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.958930654419969), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9520979134980109), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9387688286138868), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9660352858307433), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9661357152692859), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9660379136704926), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.958930654419969), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9520979134980109), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9126602155010046), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9643890726645791), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9849195487881457), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9871133344952977), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.9813227308794694), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9773489299757616)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9391945464479733), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9633912545363514), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9563006068196144), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9697019009972268), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9626388749816811), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9539353277901782), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9391945464479733), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9633912545363514), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9563006068196144), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9697019009972268), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9626388749816811), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9539353277901782), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9108404984302801), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9664114008404745), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9859085412552211), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9871133344952977), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.98684025272767), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.9823030696835358)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.9368265497597607), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9571694600729512), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9524802100124091), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9612475627035132), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9678330355473781), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9618731414101533), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.9368265497597607), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9571694600729512), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9524802100124091), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9612475627035132), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9678330355473781), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9618731414101533), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.9145415186576021), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9661398549474679), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.9760612426667575), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9813227308794694), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.98684025272767), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36', 0.989514611591274)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t36': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t31', 0.938042874780895), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t32', 0.9570082710573644), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t33', 0.9520292446367055), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t34', 0.9602304292367366), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t35', 0.9654679871617633), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t36', 0.9687344950837457), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t31', 0.938042874780895), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t32', 0.9570082710573644), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t33', 0.9520292446367055), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t34', 0.9602304292367366), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t35', 0.9654679871617633), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t36', 0.9687344950837457), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t30', 0.912488702544453), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t31', 0.9668487798273079), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t32', 0.975341424324142), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t33', 0.9773489299757616), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t34', 0.9823030696835358), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t35', 0.989514611591274)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t37': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t37', 0.9678946256910758), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t38', 0.9575510242255703), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t39', 0.9511900537357373), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t37', 0.9678946256910758), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t38', 0.9575510242255703), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t39', 0.9511900537357373), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t38', 0.9783525091392066), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t39', 0.9555963183347729)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t38': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t37', 0.9492862901829382), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t38', 0.9703131167279909), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t39', 0.9629753703589079), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t37', 0.9492862901829382), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t38', 0.9703131167279909), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t39', 0.9629753703589079), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t37', 0.9783525091392066), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t39', 0.96836331000726)], 'http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t39': [('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t37', 0.9285383458413126), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t38', 0.9482410342776318), ('http://aqiniudl.tangdou.com/202005/A7C11FD6-D846-CD4A-81BC-4A27EFB0B45C-10.mp4\t39', 0.9721080561167931), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t37', 0.9285383458413126), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t38', 0.9482410342776318), ('http://aqiniudl.tangdou.com/202004/2A6CC81A-880F-D7AA-C32D-24DDC93643BF-10.mp4\t39', 0.9721080561167931), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t37', 0.9555963183347729), ('http://aqiniudl.tangdou.com/202005/D0249C43-5190-C475-3F5D-7EE42388FA73-10.mp4\t38', 0.96836331000726)]}
#print(abc)   
print(len(abc.keys()))
cnt = 0.9765388422942951
for key in abc.keys():
    pre_fix, suffix = key.split('\t')
    self_filter = [ele for ele in abc[key] if pre_fix not in ele[0]]
    if self_filter:
        cnt += 1
int(cnt/2)/20

60


NameError: name 'cnt' is not defined

In [61]:
r = redis.Redis(host='10.42.158.47', port=6379)
key = 'video_feature_1500675673760'
abc = r.get(key)
import pickle
to_pickle = './abc.pkl'
pickle.dump(abc, open(to_pickle,'wb'),protocol=2)



# 过滤盗播视频 需要抽特征的

In [1]:
import pandas as pd 
df1 = pd.read_csv('/Users/jiangcx/Downloads/vid_for_video.csv')
df2 = pd.read_csv('/Users/jiangcx/Documents/vid_for_video(2).csv')
set1 = set(df1['vid'].tolist())
set2 = set( df2[   (df2['teach']=='教学') & (df2['star']>=3)  & (df2['duration']>=40) ]['vid'].tolist())
df3 = df2[(df2['teach']=='教学') & (df2['star']>=3)  & (df2['duration']>=40)]
myset = set2.difference(set1) #173311
df3[df3['vid'].isin(myset)].reset_index().to_csv('/Users/jiangcx/Documents/vid_for_video_filted.csv')



In [25]:
%%time
# 快速解析一天数据,bad case：训练吧
import json
import requests
import pandas as pd 

def dup(jsonbody):
    #url = "http://10.19.21.255:9005/polls/dup_detector"
    #url = "http://10.19.6.41:9003/polls/dup_detector"
    #url = "http://127.0.0.1:8000/polls/dup_detector"
    url = "http://10.10.5.41:9003/polls/dup_detector"
    url = "http://10.10.25.182:9003/polls/dup_detector"
    resp1 = requests.post(url, data=jsonbody)
    return resp1.text


df = pd.read_csv('/Users/jiangcx/Documents/jianshenba.csv')
df = df.iloc[230:]
with open('/Users/jiangcx/Documents/jianshen_pro.txt', 'a', encoding='utf-8') as f: 
    for ind, row in df.iterrows(): 
        #print(row['vid'])
        parms = {"vid":row['vid'], "title":row['title'], "uname":row['uname'] ,"teacher_id": '', 'mp3_name': '', 'createtime': row['createtime']}
        res1 = dup(parms)
        res = json.loads(res1)
        print(parms)
        if res['reprint']['img_reprint_flag'] == 1:
#             print("*"*50)
#             print(parms) 
#             print(res1)
            f.write(res1+'\n')


In [6]:
import redis
import matplotlib.pyplot as plt
import numpy as np
import json
from PIL import Image as pil_image
import urllib
import io
import cv2 
r = redis.Redis(host='10.42.158.47', port=6379)
def func(vid):
    print(vid)
    key = "url_{}".format(vid)
    abc = r.get(key)
    pic = 'http://aimg.tangdou.com' + json.loads(abc)['pic']
    print(pic)
    video = 'http://aqiniudl.tangdou.com/' + json.loads(abc)['videourl']+'-10.mp4'
    print(video)
    return pic
func(1500678648721)

1500678648721
http://aimg.tangdou.com/public/video/2021/0101/20210101235122_173559.jpg!260
http://aqiniudl.tangdou.com/202101/622A1FAE-471F-BE13-11F5-431D810C0946-10.mp4


'http://aimg.tangdou.com/public/video/2021/0101/20210101235122_173559.jpg!260'

In [1]:
# %matplotlib inline
# import redis
# import matplotlib.pyplot as plt
# import numpy as np
# import json
# from PIL import Image as pil_image
# import urllib
# import io
# import cv2 
# r = redis.Redis(host='10.42.158.47', port=6379)
# def func(vid):
#     print(vid)
#     key = "url_{}".format(vid)
#     abc = r.get(key)
#     pic = 'http://aimg.tangdou.com' + json.loads(abc)['pic']
#     print(pic)
#     video = 'http://aqiniudl.tangdou.com/' + json.loads(abc)['videourl']+'-10.mp4'
#     print(video)
#     return pic
    
# def plot_images(images, cls_true, cls_pred=None):
#     num = len(images)
#     # Create figure with 3x3 sub-plots.
#     fig, axes = plt.subplots(1, num)
#     fig.subplots_adjust(hspace=0.3, wspace=0.3)

#     for i, ax in enumerate(axes.flat):
#         # Plot image.
#         ax.imshow(images[i], cmap='binary')

#         # Show true and predicted classes.
#         if cls_pred is None:
#             xlabel = "Vid: {0}".format(cls_true[i])
#         else:
#             xlabel = "Vid: {0}, Pred: {1}".format(cls_true[i], cls_pred[i])

#         ax.set_xlabel(xlabel)
        
#         # Remove ticks from the plot.
#         ax.set_xticks([])
#         ax.set_yticks([])
        
#     # Ensure the plot is shown correctly with multiple plots
#     # in a single Notebook cell.
#     plt.show()
# def load_img(path, target_size=None):
#     try:
#         with urllib.request.urlopen(path) as url:
#             f = io.BytesIO(url.read())
#             img = pil_image.open(f)
#     except:
#         logging.error('the url cannot open {}'.format(path))
#         return None
#     if target_size is not None:
#         width_height_tuple = (target_size[1], target_size[0])
#         if img.size != width_height_tuple:
#             img = img.resize(width_height_tuple)
#     res = np.asarray(img)
#     if res.shape[-1]==4:
#         return(res[:,:,:-1])
#     else:
#         return res    
# with open('/Users/jiangcx/Documents/jianshen_pro.txt', 'r', encoding='utf-8') as f: 
#     for line in f: 
#         obj = json.loads(line.strip())
#         query_vid = obj['vid']
#         query_pic = func(query_vid)
#         img = load_img(query_pic, (224,224))
#         tmp_imgs = []
#         tmp_vids = []
#         tmp_vids.append(query_vid)
#         tmp_imgs.append(img)
#         reprint_vid = obj['reprint']['img_reprint_vidlist']
#         for x in reprint_vid:
#             reprint_pic = func(x)
#             tmp_vids.append(x)
#             i = load_img(reprint_pic, (320,320))
#             tmp_imgs.append(i)
        
#         plot_images(images=tmp_imgs, cls_true=tmp_vids)

# # 

In [120]:
1500674673128

In [4]:
import redis
import json
r = redis.Redis(host='10.42.158.47', port=6379)
def func(vid):
    print(vid)
    key = "url_{}".format(vid)
    abc = r.get(key)
    pic = 'http://aimg.tangdou.com' + json.loads(abc)['pic']
    print(pic)
    video = 'http://aqiniudl.tangdou.com/' + json.loads(abc)['videourl']+'-10.mp4'
    print(video)
    return pic
func(1500676774817)

1500676773822
http://aimg.tangdou.com/public/video/2020/0811/AC805D80FD76C371541BE9CEEF947BD1.jpg!260
http://aqiniudl.tangdou.com/202008/EA9DA7762F1A1DF2277BCEF1C4347533-10.mp4


'http://aimg.tangdou.com/public/video/2020/0811/AC805D80FD76C371541BE9CEEF947BD1.jpg!260'

In [2]:
# #coding:utf-8
# import requests
# import json
# import sys

# inputfile = sys.argv[1]
# outputfile = sys.argv[2]
# which_line = int(sys.argv[3])
# stop_line = int(sys.argv[4])


# # inputfile = '/Users/jiangcx/Documents/content_all_for_recall_new'
# # outputfile = '/Users/jiangcx/Documents/outdelete'
# # which_line = 0
# # stop_line = 10
# def dup(jsonbody):
#     url = "http://127.0.0.1:9003/polls/dup_detector"
#     #url = "http://10.10.25.182:9003/polls/dup_detector"
#     resp1 = requests.post(url, data=jsonbody)
#     return resp1.text

# with open(inputfile, 'r', encoding='utf-8') as f, open(outputfile, 'w', encoding='utf-8') as writer:
#     count = 1
#     for line in f:
#         if count >= which_line and count <= stop_line:
            
#             tmp = json.loads(line)
#             tmp['title'] = '000'
#             print(tmp)
#             res = dup(tmp)
#             print(res)
#             writer.write(res+'\n')
#         count += 1

In [50]:
import requests
import json
import sys
def eschecker(vids):
    vid_list = [vids[i:i + 5000] for i in range(0, len(vids), 5000)]
    esvids = []
    c = 0
    for v_list in vid_list:
        url = "http://10.19.87.8:9221/portrait_video_v1/video/_search"
        payload = {"size": 5000, "query": {"terms": {"id": v_list}},
                   "_source": ['vid', 'talentstar', 'content_teach']}

        payload = json.dumps(payload)
        headers = {
            'Content-Type': "application/json",
            'cache-control': "no-cache"
        }
        response = requests.request("POST", url, data=payload, headers=headers)
        reses = []
        json1 = response.json()['hits']['hits']
        for x in json1:
            my_dict = {}
            res = x['_source']
            if 'vid' in res:
                vid = res['vid']
            else:
                continue
            try:
                talentstar = res['talentstar']
                tmp_teach = res['content_teach']
                if 'tagid' in tmp_teach:
                    tagid = int(tmp_teach['tagid'])
                else:
                    tagid = 0
                if talentstar >=3 and tagid==362:
                    new = {}
                    new['id'] = vid
                    new['talentstar'] = talentstar
                    newline = json.dumps(new, ensure_ascii=False)
                    reses.append(newline)
            except:
                print(vid)
            # if 'title' in res:
            #      title = res['title']
            #     if 'dj' in title.lower():
            #         res = (vid, title,content_mp3)
            #         reses.append(res)
            # else:
            #     c +=1
        esvids.extend(reses)
    return esvids
vids = []
with open('/Users/jiangcx/Documents/content_all_for_recall_new', 'r', encoding='utf-8') as f:
    for line in f:
        line = json.loads(line)
        vid = line['vid']
        vids.append(vid)
        
results = eschecker(vids)
print(len(results))

451
450
62193


In [6]:
9%10 

9

In [59]:
import requests
import json
import sys
def eschecker(vids):
    vid_list = [vids[i:i + 5000] for i in range(0, len(vids), 5000)]
    for v_list in vid_list:
        url = "http://10.19.87.8:9221/portrait_video_v1/video/_search"
        payload = {"size": 5000, "query": {"terms": {"id": v_list}},
                   "_source": ['vid', 'talentstar', 'content_teach']}

        payload = json.dumps(payload)
        headers = {
            'Content-Type': "application/json",
            'cache-control': "no-cache"
        }
        response = requests.request("POST", url, data=payload, headers=headers)
        json1 = response.json()['hits']['hits']
        for x in json1:
            res = x['_source']
            if 'vid' in res:
                vid = res['vid']
            else:
                continue
            try:
                talentstar = res['talentstar']
                tmp_teach = res['content_teach']
                if 'tagid' in tmp_teach:
                    tagid = int(tmp_teach['tagid'])
                else:
                    tagid = 0
                if talentstar >=3 and tagid==362:
                    return True
            except:
                return False
    return False

results = eschecker(1500670829617)
print(results)

False


In [ ]:
import pandas 